In [1]:
import pandas as pd
import os
import requests
import json
#import numpy as np


import glob
from pathlib import Path
import csv

import quandl
import yfinance as yf

from dotenv import load_dotenv
load_dotenv()

True

In [22]:
nas_api_key = os.environ.get('NASDAQ_API_KEY')

iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

iex_base_url = 'https://cloud.iexapis.com/stable/'
iex_sandbox_url = 'https://sandbox.iexapis.com/stable/'

iex_real_token = os.environ.get('IEX_API_KEY')
test_token = os.environ.get('IEX_TEST_API_KEY')
test_resp = requests.get(iex_base_url + 'status')
test_resp

<Response [200]>

In [3]:
## 
## https://data.nasdaq.com/api/v3/datasets/FINRA/FNYX_GME.csv?api_key=ZEziiYbLDu8PEboZzd9x
## https://data.nasdaq.com/api/v3/datasets/FINRA/FNYX_GME.csv?api_key=ZEziiYbLDu8PEboZzd9x

In [4]:
base_url = 'https://data.nasdaq.com/api/v3/datasets/FINRA/'

In [5]:
#gme_test = quandl.get("FINRA/FNRA_GME", authtoken=nas_api_key)
gme_test1= quandl.get("FINRA/FNSQ_GME", authtoken=nas_api_key)   ## Nasdaq
gme_test2= quandl.get("FINRA/FNYX_GME", authtoken=nas_api_key)   ## NYSE

In [6]:
gme_test1 = gme_test1.rename(columns={'ShortVolume':'ShortVolumeNSDQ','TotalVolume':'TotalVolumeNSDQ'})
gme_testNSDQ = gme_test1.drop(columns={'ShortExemptVolume'})
gme_testNSDQ

,ShortVolumeNSDQ,TotalVolumeNSDQ
Date,,
2013-04-01,1123726.0,2331585.0
2013-04-02,357350.0,1119089.0
2013-04-03,369376.0,1331007.0
2013-04-04,580145.0,1365674.0
2013-04-05,261149.0,524296.0
...,...,...
2021-10-01,215235.0,381883.0
2021-10-04,206320.0,323567.0
2021-10-05,174076.0,306657.0


In [7]:
gme_test2 = gme_test2.rename(columns={'ShortVolume':'ShortVolumeNYSE','TotalVolume':'TotalVolumeNYSE'})
gme_testNYSE = gme_test2.drop(columns={'ShortExemptVolume'})
gme_testNYSE

,ShortVolumeNYSE,TotalVolumeNYSE
Date,,
2013-04-01,204253.0,318290.0
2013-04-02,47973.0,100342.0
2013-04-03,31835.0,51906.0
2013-04-04,38855.0,65558.0
2013-04-05,58309.0,66865.0
...,...,...
2021-10-01,53111.0,96005.0
2021-10-04,32041.0,73299.0
2021-10-05,49739.0,92450.0


In [8]:
gme_merge = gme_testNSDQ.merge(gme_testNYSE, how='inner', right_index=True, left_index=True)
gme_merge

,ShortVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,TotalVolumeNYSE
Date,,,,
2013-04-01,1123726.0,2331585.0,204253.0,318290.0
2013-04-02,357350.0,1119089.0,47973.0,100342.0
2013-04-03,369376.0,1331007.0,31835.0,51906.0
2013-04-04,580145.0,1365674.0,38855.0,65558.0
2013-04-05,261149.0,524296.0,58309.0,66865.0
...,...,...,...,...
2021-10-01,215235.0,381883.0,53111.0,96005.0
2021-10-04,206320.0,323567.0,32041.0,73299.0
2021-10-05,174076.0,306657.0,49739.0,92450.0


In [9]:
# amc_test2= quandl.get("FINRA/FNSQ_AMC", authtoken=nas_api_key)
# amc_test3= quandl.get("FINRA/FNYX_AMC", authtoken=nas_api_key)

In [10]:
def import_df(csv_path):
    df = pd.read_csv(
    Path(csv_path),
    infer_datetime_format=True,
    parse_dates=True,
    index_col="Date",
    )
    return df 

In [11]:
gme_df = import_df('../Resources/gme_df.csv')
gme_df

,GME_QUANTITY_FAILS,close,volume,price_range,daily_change,fail_volume_pct,fail_float_pct
Date,,,,,,,
2020-09-14,398676,6.91,10373189,0.4100,0.090,3.843331,0.645526
2020-09-15,267777,7.39,5806696,0.6980,-0.340,4.611521,0.433577
2020-09-16,884485,8.71,19943923,2.1600,-1.340,4.434860,1.432135
2020-09-17,598737,9.20,17777853,1.6200,-0.520,3.367881,0.969459
2020-09-18,847632,9.73,17437164,1.0000,-0.230,4.861066,1.372463
...,...,...,...,...,...,...,...
2021-09-08,96,207.70,5325598,22.3377,2.080,0.001803,0.000155
2021-09-09,544,204.09,7569339,25.0900,-15.258,0.007187,0.000881
2021-09-10,4836,194.87,3088838,13.3900,5.963,0.156564,0.007830


In [12]:
gme_merge2 = gme_df.merge(gme_merge, how='inner', right_index=True, left_index=True)
gme_merge2

,GME_QUANTITY_FAILS,close,volume,price_range,daily_change,fail_volume_pct,fail_float_pct,ShortVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,TotalVolumeNYSE
Date,,,,,,,,,,,
2020-09-14,398676,6.91,10373189,0.4100,0.090,3.843331,0.645526,1975257.0,3188726.0,661406.0,1110512.0
2020-09-15,267777,7.39,5806696,0.6980,-0.340,4.611521,0.433577,1202584.0,2023271.0,448102.0,830595.0
2020-09-16,884485,8.71,19943923,2.1600,-1.340,4.434860,1.432135,4033057.0,6985205.0,1539765.0,3018080.0
2020-09-17,598737,9.20,17777853,1.6200,-0.520,3.367881,0.969459,3468367.0,6296379.0,1603041.0,2746421.0
2020-09-18,847632,9.73,17437164,1.0000,-0.230,4.861066,1.372463,2612609.0,4564957.0,875288.0,1679697.0
...,...,...,...,...,...,...,...,...,...,...,...
2021-09-08,96,207.70,5325598,22.3377,2.080,0.001803,0.000155,802472.0,1376908.0,202733.0,399528.0
2021-09-09,544,204.09,7569339,25.0900,-15.258,0.007187,0.000881,1440885.0,2293702.0,324198.0,663719.0
2021-09-10,4836,194.87,3088838,13.3900,5.963,0.156564,0.007830,547904.0,925400.0,151874.0,278469.0


In [13]:
gme_merge2['Add_NYSE_NSDQ_Volume'] = gme_merge2['TotalVolumeNSDQ'] + gme_merge2['TotalVolumeNYSE']

In [14]:
gme_merge2

,GME_QUANTITY_FAILS,close,volume,price_range,daily_change,fail_volume_pct,fail_float_pct,ShortVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,TotalVolumeNYSE,Add_NYSE_NSDQ_Volume
Date,,,,,,,,,,,,
2020-09-14,398676,6.91,10373189,0.4100,0.090,3.843331,0.645526,1975257.0,3188726.0,661406.0,1110512.0,4299238.0
2020-09-15,267777,7.39,5806696,0.6980,-0.340,4.611521,0.433577,1202584.0,2023271.0,448102.0,830595.0,2853866.0
2020-09-16,884485,8.71,19943923,2.1600,-1.340,4.434860,1.432135,4033057.0,6985205.0,1539765.0,3018080.0,10003285.0
2020-09-17,598737,9.20,17777853,1.6200,-0.520,3.367881,0.969459,3468367.0,6296379.0,1603041.0,2746421.0,9042800.0
2020-09-18,847632,9.73,17437164,1.0000,-0.230,4.861066,1.372463,2612609.0,4564957.0,875288.0,1679697.0,6244654.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-08,96,207.70,5325598,22.3377,2.080,0.001803,0.000155,802472.0,1376908.0,202733.0,399528.0,1776436.0
2021-09-09,544,204.09,7569339,25.0900,-15.258,0.007187,0.000881,1440885.0,2293702.0,324198.0,663719.0,2957421.0
2021-09-10,4836,194.87,3088838,13.3900,5.963,0.156564,0.007830,547904.0,925400.0,151874.0,278469.0,1203869.0


In [15]:
## Rebuild gme_df using IEX to check volume 
## Volume stats do not make sense 

In [23]:
def get_daily_data_iex(stock_ticker, chart_range='14m'):

    resp_data = requests.get(iex_sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    df = df[['close','open','low','high','volume']]
    #df.rename(columns={'close':stock_ticker},inplace=True)
    
    return df

def get_all_data_iex(stock_ticker, chart_range='14m'):

    resp_data = requests.get(iex_sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    #df = df[['close','open','low','high','volume']]
    #df.rename(columns={'close':stock_ticker},inplace=True)
    
    return df

def get_all_data_iex_real(stock_ticker, chart_range='14m'):

    resp_data = requests.get(iex_base_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+iex_real_token)
    df = pd.DataFrame(resp_data.json())

    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
    
    #df = df[['close','open','low','high','volume']]
    #df.rename(columns={'close':stock_ticker},inplace=True)
    
    return df

def get_iex_data(stock_ticker, chart_range='14m',api_token=False)
    if api_token == True:     # If api_token = True , use Real IEX Api 
        resp_data = requests.get(iex_base_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+iex_real_token)
        df = pd.DataFrame(resp_data.json())
    else:
        resp_data = requests.get(iex_sandbox_url+'stock/'+stock_ticker+'/chart/'+chart_range+'?token='+test_token)
        df = pd.DataFrame(resp_data.json())
        
    df.rename(columns={'date':'Date'},inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date',inplace=True)
        

In [24]:
# gme_iex_df = get_all_data_iex_real('GME')
# gme_iex_df

,close,high,low,open,symbol,volume,id,key,subkey,updated,...,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent
Date,,,,,,,,,,,,,,,,,,,,,
2020-08-10,4.33,4.5700,4.1800,4.20,GME,4561772,HISTORICAL_PRICES,GME,,1606830572000,...,4.1800,4561772,4.20,4.33,4.5700,4.1800,4561772,"Aug 10, 20",0.00,0.0000
2020-08-11,4.35,4.5700,4.3400,4.43,GME,3139105,HISTORICAL_PRICES,GME,,1606830572000,...,4.3400,3139105,4.43,4.35,4.5700,4.3400,3139105,"Aug 11, 20",0.02,0.0046
2020-08-12,4.52,4.6300,4.3600,4.40,GME,3057594,HISTORICAL_PRICES,GME,,1606830572000,...,4.3600,3057594,4.40,4.52,4.6300,4.3600,3057594,"Aug 12, 20",0.17,0.0391
2020-08-13,4.64,4.7100,4.4966,4.52,GME,2128313,HISTORICAL_PRICES,GME,,1606830572000,...,4.4966,2128313,4.52,4.64,4.7100,4.4966,2128313,"Aug 13, 20",0.12,0.0265
2020-08-14,4.75,4.8300,4.5450,4.60,GME,3474692,HISTORICAL_PRICES,GME,,1606830572000,...,4.5450,3474692,4.60,4.75,4.8300,4.5450,3474692,"Aug 14, 20",0.11,0.0237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30,175.47,185.4499,166.7900,175.00,GME,3177453,HISTORICAL_PRICES,GME,,1633049411000,...,166.7900,3177453,175.00,175.47,185.4499,166.7900,3177453,"Sep 30, 21",-0.45,-0.0026
2021-10-01,176.91,179.4900,172.2500,179.40,GME,1551139,HISTORICAL_PRICES,GME,,1633137549000,...,172.2500,1551139,179.40,176.91,179.4900,172.2500,1551139,"Oct 1, 21",1.44,0.0082
2021-10-04,171.36,176.9500,170.7710,175.95,GME,1195738,HISTORICAL_PRICES,GME,,1633399406000,...,170.7710,1195738,175.95,171.36,176.9500,170.7710,1195738,"Oct 4, 21",-5.55,-0.0314


In [25]:
# gme_merge_iex = gme_iex_df.merge(gme_merge, how='inner', right_index=True, left_index=True)
# gme_merge_iex

,close,high,low,open,symbol,volume,id,key,subkey,updated,...,fHigh,fLow,fVolume,label,change,changePercent,ShortVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,TotalVolumeNYSE
Date,,,,,,,,,,,,,,,,,,,,,
2020-08-10,4.33,4.5700,4.1800,4.20,GME,4561772,HISTORICAL_PRICES,GME,,1606830572000,...,4.5700,4.1800,4561772,"Aug 10, 20",0.00,0.0000,399383.0,1103863.0,110769.0,324593.0
2020-08-11,4.35,4.5700,4.3400,4.43,GME,3139105,HISTORICAL_PRICES,GME,,1606830572000,...,4.5700,4.3400,3139105,"Aug 11, 20",0.02,0.0046,394763.0,999332.0,89257.0,386917.0
2020-08-12,4.52,4.6300,4.3600,4.40,GME,3057594,HISTORICAL_PRICES,GME,,1606830572000,...,4.6300,4.3600,3057594,"Aug 12, 20",0.17,0.0391,262255.0,815133.0,87540.0,197584.0
2020-08-13,4.64,4.7100,4.4966,4.52,GME,2128313,HISTORICAL_PRICES,GME,,1606830572000,...,4.7100,4.4966,2128313,"Aug 13, 20",0.12,0.0265,350657.0,683277.0,32564.0,117184.0
2020-08-14,4.75,4.8300,4.5450,4.60,GME,3474692,HISTORICAL_PRICES,GME,,1606830572000,...,4.8300,4.5450,3474692,"Aug 14, 20",0.11,0.0237,412250.0,1032628.0,115467.0,299058.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30,175.47,185.4499,166.7900,175.00,GME,3177453,HISTORICAL_PRICES,GME,,1633049411000,...,185.4499,166.7900,3177453,"Sep 30, 21",-0.45,-0.0026,578990.0,920171.0,105664.0,208036.0
2021-10-01,176.91,179.4900,172.2500,179.40,GME,1551139,HISTORICAL_PRICES,GME,,1633137549000,...,179.4900,172.2500,1551139,"Oct 1, 21",1.44,0.0082,215235.0,381883.0,53111.0,96005.0
2021-10-04,171.36,176.9500,170.7710,175.95,GME,1195738,HISTORICAL_PRICES,GME,,1633399406000,...,176.9500,170.7710,1195738,"Oct 4, 21",-5.55,-0.0314,206320.0,323567.0,32041.0,73299.0


In [26]:
#gme_iex_df.to_csv('../Resources/gme_iex_df.csv') 

## import CSV when needed


In [34]:
def get_ticker_info_yf(symbol):
    str_symbol = str(symbol)
    ticker = yf.Ticker(str_symbol)
    ticker_info = ticker.info
    try:
        sharesOutstanding = ticker_info['sharesOutstanding']
    except KeyError:
        sharesOutstanding = 0
    try:
        floatShares = ticker_info['floatShares']
    except KeyError:
        floatShares = 0
#     dictionary = {
#         'sharesOutstanding' : sharesOutstanding,
#         'floatShares' : floatShares
#     }
    return ticker_info

In [35]:
# ?filter=symbol,volume,lastSalePrice

In [40]:
# gme_yf_info = get_ticker_info('GME')
# gme_yf_info